# Rename positrack2 files

The positrack2 software has the following name scheme: mouse-date-time.positrack2
We want to have mouse-date_01.positrack2, mouse-date_02.positrack2, etc
We write a function that works on a single session.

It will also copy the .mp4 file that goes with it to mouse-date_01.mp4, etc.

In [1]:
%load_ext autoreload
%autoreload 2
import glob
from datetime import datetime
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from autopipy.project import Project
from spikeA.Session import Kilosort_session
from spikeA.Animal_pose import Animal_pose

In [2]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
load_parameters_from_files_project(sSesList)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 44 testing sessions in the list
See myProject and sSesList objects


100%|██████████████████████████████████████████| 44/44 [00:00<00:00, 564.90it/s]


In [4]:

# find sessions using positrack2
toChangeSessions = []
for ses in sSesList:
    ses.load_parameters_from_files() 
    if  glob.glob(ses.path+"/*.positrack2"): # any positrack2 files
        print(ses.name)
        # check if the correct file names are already there
        #if not os.path.exists(ses.path+"/"+ses.trial_names[0]+".positrack2"):
        toChangeSessions.append(ses)

print("{} sessions to change".format(len(toChangeSessions)))


TYY9524-16082021-0106
TYY9524-18082021-0106
TYY9524-20082021-0106
TYY9524-22082021-0106
TYY9524-23082021-0105
TYY9524-25082021-0106
TYY5622-07092021-0106
TYY5622-17092021-0106
TYY5622-19092021-0106
TYY5622-20092021-0106
mn9686-20102021-0106
mn9686-26102021-0106
mn9686-27102021-0106
mn9686-28102021-0107
mn9686-29102021-0106
mn9686-01112021-0106
16 sessions to change


Select a session to process

In [83]:
#ses = sSesList[43]
ses = toChangeSessions[-16]
print(ses.name)
print(ses.path)

TYY9524-16082021-0106
/adata/projects/autopi_ca1/TYY9524/TYY9524-16082021-0106


Create a datetime object for each file name

In [79]:
posFiles = glob.glob(ses.path+"/*-*-*.positrack2")
shortPosFiles = [fn.split(sep="/")[-1] for fn in posFiles]
shortMp4Files = [ fn.replace("positrack2","mp4")  for fn in shortPosFiles]
dates = [fn.split(sep="-")[1] for fn in shortPosFiles]
times = [fn.split(sep="-")[2].split(sep=".")[0] for fn in shortPosFiles]
dt = [ datetime.strptime(d[0]+d[1],"%d%m%Y%H%M%S") for d in zip(dates,times)]
for d in zip(shortPosFiles,shortMp4Files,dt):
    print(d)

('TYY9524-18082021-184430.positrack2', 'TYY9524-18082021-184430.mp4', datetime.datetime(2021, 8, 18, 18, 44, 30))
('TYY9524-18082021-201813.positrack2', 'TYY9524-18082021-201813.mp4', datetime.datetime(2021, 8, 18, 20, 18, 13))
('TYY9524-18082021-163620.positrack2', 'TYY9524-18082021-163620.mp4', datetime.datetime(2021, 8, 18, 16, 36, 20))
('TYY9524-18082021-191542.positrack2', 'TYY9524-18082021-191542.mp4', datetime.datetime(2021, 8, 18, 19, 15, 42))
('TYY9524-18082021-170725.positrack2', 'TYY9524-18082021-170725.mp4', datetime.datetime(2021, 8, 18, 17, 7, 25))
('TYY9524-18082021-194641.positrack2', 'TYY9524-18082021-194641.mp4', datetime.datetime(2021, 8, 18, 19, 46, 41))


We now sort our datetime. More specifically we get the index of the datetime object once sorted.

Then we can just loop over the indices and use enumerate to get the trial number.

In [80]:
indices = sorted(range(len(dt)), key=lambda k: dt[k])
print(indices)
for e,i in enumerate(indices):
    trial_no=e+1
    mouse=shortPosFiles[i].split("-")[0]
    date=shortPosFiles[i].split("-")[1]
    newFileName="{}-{}_{:02d}.positrack2".format(mouse,date,trial_no)
    print(dt[i],shortPosFiles[i],trial_no,newFileName)
    

[2, 4, 0, 3, 5, 1]
2021-08-18 16:36:20 TYY9524-18082021-163620.positrack2 1 TYY9524-18082021_01.positrack2
2021-08-18 17:07:25 TYY9524-18082021-170725.positrack2 2 TYY9524-18082021_02.positrack2
2021-08-18 18:44:30 TYY9524-18082021-184430.positrack2 3 TYY9524-18082021_03.positrack2
2021-08-18 19:15:42 TYY9524-18082021-191542.positrack2 4 TYY9524-18082021_04.positrack2
2021-08-18 19:46:41 TYY9524-18082021-194641.positrack2 5 TYY9524-18082021_05.positrack2
2021-08-18 20:18:13 TYY9524-18082021-201813.positrack2 6 TYY9524-18082021_06.positrack2


If you are happy with the new file names, run the following.

In [81]:
indices = sorted(range(len(dt)), key=lambda k: dt[k])
print(indices)

[2, 4, 0, 3, 5, 1]


In [82]:
for e,i in enumerate(indices):
    trial_no=e+1
    mouse=shortPosFiles[i].split("-")[0]
    date=shortPosFiles[i].split("-")[1]
    for fileList in [shortPosFiles, shortMp4Files]:
        filename, ext = os.path.splitext(fileList[i])
        
        newPosFileName="{}-{}_{:02d}{}".format(mouse,date,trial_no,ext)
        old = "{}/{}".format(ses.path,fileList[i])
        new = "{}/{}".format(ses.path,newPosFileName)
        if os.path.exists(new):
            print(new,"exists, delete it manually.")
            break
        print(old)
        print(new)
        shutil.copyfile(old,new)

/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021-163620.positrack2
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021_01.positrack2
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021-163620.mp4
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021_01.mp4
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021-170725.positrack2
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021_02.positrack2
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021-170725.mp4
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021_02.mp4
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021-184430.positrack2
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021_03.positrack2
/adata/projects/autopi_ca1/TYY9524/TYY9524-18082021-0106/TYY9524-18082021-184430.mp4
/adata/projects/autopi_ca1/TYY9524/TYY9524-